In [936]:
# It contains all operations for building a machine learning model

Importing files/data from analysis.ipynb file for creating model

In [937]:
import pprint

In [938]:
import nltk
import pandas as pd
nltk.download('punkt')
import import_ipynb
pd.set_option('display.max_rows', None)
from analysis import *
import pickle
import requests
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TUSHAR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [939]:
data.head()

,repo_name,star,fork,watch,issue,tags,description,contributers,license,repo_url,most_used_lang,tag_ratio,repo_desc,language,desc_to_num,repo_to_num
0,keras,47900,18100,2100,2940,"['deep-learning', 'tensorflow', 'neural-networ...",deep learning for humans,49,1,https://github.com/keras-team/keras,python,9.439798,deep learning for humans,[python],571,73
1,scikit-learn,40300,19600,2200,1505,"['machine-learning', 'python', 'statistics', '...",scikit learn machine learning in python,108,1,https://github.com/scikit-learn/scikit-learn,python,6.048803,scikit learn machine learning in python,[python],307,33
2,PythonDataScienceHandbook,23100,9900,1500,65,"['scikit-learn', 'numpy', 'python', 'jupyter-n...",python data science handbook full text in jup...,0,1,https://github.com/jakevdp/PythonDataScienceHa...,jupyter notebook,2.463054,python data science handbook full text in jup...,"[jupyter, notebook]",859,0
3,Probabilistic-Programming-and-Bayesian-Methods...,21000,6600,1400,127,"['bayesian-methods', 'pymc', 'mathematical-ana...",aka bayesian methods for hackers an introdu...,0,1,https://github.com/CamDavidsonPilon/Probabilis...,jupyter notebook,1.752778,aka bayesian methods for hackers an introdu...,"[jupyter, notebook]",6863,0
4,Data-Science--Cheat-Sheet,18400,8200,1500,7,[],cheat sheets,0,0,https://github.com/abhat222/Data-Science--Chea...,no language,0.000000,cheat sheets,"[no, language]",902,0


Description is also play a role in the popularity of your repos, so we need to clean  `description` tuple by removing punctuations and creating word counts.

In [940]:
data['repo_desc']=data['description']
data['description']=data['description'].str.replace('\W'," ")
data['description']=data['description'].str.lower()

In [941]:
data['most_used_lang']=data['most_used_lang'].str.replace('\W'," ")
data['most_used_lang']=data['most_used_lang'].str.lower()

In [942]:
data['language']=data['most_used_lang'].str.split()
vocabulary = []
for words in data['language']:
    for word in words:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

In [943]:
word_counts_for_language = {unique_word: [0] * len(data['language']) for unique_word in vocabulary}

for index, words in enumerate(data['language']):
    for word in words:
        word_counts_for_language[word][index] += 1

language_col=pd.DataFrame(word_counts_for_language)

Tokenizing description 

In [944]:
text=[]
for index,row in data.iterrows():
    tokens=nltk.word_tokenize(row['description'])
    text.append(tokens)

In [945]:
from nltk.corpus import stopwords
stopword=stopwords.words('english')

To get numerical form of description , we create word dict according to their frequency i.e. the word which occurs more would be have low index value and removing words which has less `freq`(threshold=1)

In [946]:
def txt_to_num(text, threshold_rarewords = 1):
    
    # removing sublist from text list
    if len(text) > 1:
        text_list = [item for sublist in text for item in sublist]
    else:
        text_list = text
    
    # get word freuqncy
    freq_word = nltk.FreqDist(text_list)


    df = pd.DataFrame.from_dict(freq_word, orient='index')
    df.columns = ['freq']
      

    df.sort_values(by=['freq'], ascending=False, inplace=True)
  
    # Add word index
    number_of_words = df.shape[0]
    for word in df.index:
        if word in stopword:
            df.loc[word]['freq']=0
    df.sort_values('freq',inplace=True,ascending=False)
    
    df['word_index'] = list(np.arange(number_of_words)+1)
    
    # indexing 0 for rare words 
    frequency = df['freq'].values
    word_index = df['word_index'].values
    mask = frequency <= threshold_rarewords
    word_index[mask] = 0
    df['word_index'] =  word_index
    

    word_dict = df['word_index'].to_dict()

    #dict for word into num
    text_num = []
    for string in text:
        string_numbers = [word_dict[word] for word in string]
        text_num.append(string_numbers)  
    
    return (text_num,word_dict)

In [947]:
desc_num,word_dict=txt_to_num(text)
txt=[sum(x)  for x in desc_num]
data['desc_to_num']=txt

In [948]:
repo_text=[]
for index,row in data.iterrows():
    tokens=nltk.word_tokenize(row['repo_name'])
    repo_text.append(tokens)

In [949]:
repo_num,word_dict=txt_to_num(repo_text)
txt=[sum(x)  for x in repo_num]
data['repo_to_num']=txt

In [950]:
data.head()

,repo_name,star,fork,watch,issue,tags,description,contributers,license,repo_url,most_used_lang,tag_ratio,repo_desc,language,desc_to_num,repo_to_num
0,keras,47900,18100,2100,2940,"['deep-learning', 'tensorflow', 'neural-networ...",deep learning for humans,49,1,https://github.com/keras-team/keras,python,9.439798,deep learning for humans,[python],571,73
1,scikit-learn,40300,19600,2200,1505,"['machine-learning', 'python', 'statistics', '...",scikit learn machine learning in python,108,1,https://github.com/scikit-learn/scikit-learn,python,6.048803,scikit learn machine learning in python,[python],307,33
2,PythonDataScienceHandbook,23100,9900,1500,65,"['scikit-learn', 'numpy', 'python', 'jupyter-n...",python data science handbook full text in jup...,0,1,https://github.com/jakevdp/PythonDataScienceHa...,jupyter notebook,2.463054,python data science handbook full text in jup...,"[jupyter, notebook]",859,0
3,Probabilistic-Programming-and-Bayesian-Methods...,21000,6600,1400,127,"['bayesian-methods', 'pymc', 'mathematical-ana...",aka bayesian methods for hackers an introdu...,0,1,https://github.com/CamDavidsonPilon/Probabilis...,jupyter notebook,1.752778,aka bayesian methods for hackers an introdu...,"[jupyter, notebook]",6863,0
4,Data-Science--Cheat-Sheet,18400,8200,1500,7,[],cheat sheets,0,0,https://github.com/abhat222/Data-Science--Chea...,no language,0.000000,cheat sheets,"[no, language]",902,0


From above we can see that their are some numerical values and stopwords too, so we need to remove them from vocabulary list

In [951]:
training_set=pd.concat([data,language_col],axis=1)
training_set.columns

Index(['repo_name', 'star', 'fork', 'watch', 'issue', 'tags', 'description',
       'contributers', 'license', 'repo_url', 'most_used_lang', 'tag_ratio',
       'repo_desc', 'language', 'desc_to_num', 'repo_to_num', 'pure',
       'processing', 'php', 'hcl', 'java', 'r', 'swift', 'jupyter', 'makefile',
       'vue', 'qml', 'eagle', 'shell', 'jsonnet', 'pascal', 'kotlin', 'roff',
       'data', 'lua', 'script', 'coffeescript', 'dart', 'other', 'ruby',
       'typescript', 'actionscript', 'powershell', 'julia', 'assembly',
       'freemarker', 'dockerfile', 'groovy', 'haskell', 'notebook',
       'objective', 'ocaml', 'python', 'go', 'clojure', 'matlab', 'css', 'tex',
       'c', 'xslt', 'language', 'rust', 'purebasic', 'javascript', 'vim',
       'perl', 'no', 'shaderlab', 'elixir', 'html', 'scala'],
      dtype='object')

In [952]:
training_set.drop(['repo_name','tags','description','repo_url','language','no','repo_desc','most_used_lang'],axis=1,inplace=True)

In [953]:
training_set.columns

Index(['star', 'fork', 'watch', 'issue', 'contributers', 'license',
       'tag_ratio', 'desc_to_num', 'repo_to_num', 'pure', 'processing', 'php',
       'hcl', 'java', 'r', 'swift', 'jupyter', 'makefile', 'vue', 'qml',
       'eagle', 'shell', 'jsonnet', 'pascal', 'kotlin', 'roff', 'data', 'lua',
       'script', 'coffeescript', 'dart', 'other', 'ruby', 'typescript',
       'actionscript', 'powershell', 'julia', 'assembly', 'freemarker',
       'dockerfile', 'groovy', 'haskell', 'notebook', 'objective', 'ocaml',
       'python', 'go', 'clojure', 'matlab', 'css', 'tex', 'c', 'xslt', 'rust',
       'purebasic', 'javascript', 'vim', 'perl', 'shaderlab', 'elixir', 'html',
       'scala'],
      dtype='object')

Training model

In [954]:
## LR can be used:
# gradient descent
# reandom forest 
# NN regression (Relu)
# Lasso regression
# Ridge regression
# Elastic netregression


In [955]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(training_set.iloc[:,1:],training_set.iloc[:,0], test_size=0.2, random_state=22)

In [956]:
ac=0

Gradient Descent

In [957]:
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor(loss='squared_loss',alpha=0.001,max_iter=3500)
sgd.fit(X_train, y_train)
predictions = sgd.predict(X_test)
mse = mean_squared_error(y_test, predictions, squared=False)
print(mse)
accu=r2_score(y_test, predictions)
print("R2 score: ",accu)
if accu>ac:
    model=sgd
    ac=accu

1.9214192725600044e+16
R2 score:  -1.4262102763240325e+24


Normal Equation

In [958]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
mse = mean_squared_error(y_test, predictions, squared=False)
print(mse)
accu=r2_score(y_test, predictions)
print("R2 score: ",accu)
if accu>ac:
    model=lr
    ac=accu

5678.2211470383445
R2 score:  0.8754442211063735


Random forest

In [959]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=20, random_state=1)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
mse = mean_squared_error(y_test, predictions, squared=False)
print(mse)
accu=r2_score(y_test, predictions)
print("R2 score: ",accu)
if accu>ac:
    model=rf
    ac=accu

5073.451810550807
R2 score:  0.9005633795326237


In [960]:
# from numpy import loadtxt
# from keras import Sequential
# from keras.layers import Dense
# X = data[['fork','watch','issue','contributers','license','tag_ratio','desc_to_num','repo_to_num']]
# y = data['star']

# model = Sequential()
# model.add(Dense(8, input_dim=8, activation='relu'))
# model.add(Dense(1, activation='relu'))


# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit(X, y, epochs=150)

# _, accuracy = model.evaluate(X, y)
# print('Accuracy: %.2f' % (accuracy*100))

Lasso


In [961]:
from sklearn import linear_model
las = linear_model.Lasso(alpha=0.01,max_iter=2000)
las.fit(X_train, y_train)
predictions = las.predict(X_test)
mse = mean_squared_error(y_test, predictions, squared=False)
print(mse)
accu=r2_score(y_test, predictions)
print("R2 score: ",accu)
if accu>ac:
    model=las
    ac=accu

5678.156370378133
R2 score:  0.8754470629334656


Ridge

In [962]:
from sklearn import linear_model
las = linear_model.Ridge(alpha=0.001,max_iter=3000)
las.fit(X_train, y_train)
predictions = las.predict(X_test)
mse = mean_squared_error(y_test, predictions, squared=False)
print(mse)
accu=r2_score(y_test, predictions)
print("R2 score: ",accu)
if accu>ac:
    model=las
    ac=accu

5678.255485031909
R2 score:  0.875442714645686


Elastic netregression

In [963]:
from sklearn.linear_model import ElasticNet
elsnr = ElasticNet(alpha=0.1,max_iter=3000)
elsnr.fit(X_train, y_train)
predictions = elsnr.predict(X_test)
mse = mean_squared_error(y_test, predictions, squared=False)
print(mse)
accu=r2_score(y_test, predictions)
print("R2 score: ",accu)
if accu>ac:
    model=elsnr
    ac=accu

6014.400947498268
R2 score:  0.8602589410834716


In [964]:
save_model=pickle.dumps(model)

In [965]:
user_name=input("Enter username:")
repo_name=input("enter repo name:")

In [966]:
header={'Accept':'application/vnd.github.mercy-preview+json',
'visibility':'PUBLIC'
}
request=requests.get('https://api.github.com/repos/'+user_name+'/'+repo_name,headers=header)
data=request.json()
keys=['forks','watchers','open_issues_count','topics','description','language']
dataf=[data[k] for k in keys]
if str(data['license']).lower()=='none' or data['license']:
    license=0
else:
    license=1

request=requests.get('https://api.github.com/repos/'+user_name+'/'+repo_name+'/contributors',headers=header)
dat=request.json()
contributors=len(dat)
dataf.append(contributors)
dataf.append(license)
df=pd.DataFrame([dataf],columns=['fork','watch','issue','tags','description','most_used_lang','contributers','license'])
def perc(tags):
    sum_of_perc=0
    for tag in tags:
        if tag in store_tag:
            sum_of_perc+=(store_tag[tag]/vocabulary)
   
    return (sum_of_perc*100)

df['tag_ratio']=df['tags'].apply(perc).astype(float)
text=nltk.word_tokenize(df['description'][0])
tex=nltk.word_tokenize(repo_name)
def string_num(text):
    string_numbers=0
    for string in text:
        if string in word_dict:
            string_numbers+=word_dict[string]
    return string_numbers
df['desc_to_num']=string_num(text)
df['repo_to_num']=string_num(tex)
word_counts_for_language = {unique_word: [0] for unique_word in vocabulary}
df['most_used_lang']=df['most_used_lang'].str.split()
for word in df['most_used_lang']:
    if word[0] in word_counts_for_language:
        word_counts_for_language[word.lower()][index] += 1

language_col=pd.DataFrame(word_counts_for_language)
test_set=pd.concat([df,language_col],axis=1)
test_set.drop(['tags','description','language','no','most_used_lang'],axis=1,inplace=True)
predict_model = pickle.loads(save_model) 
print("your repo Popularity:",predict_model.predict(test_set)[0])

your repo Popularity: 11.29
